In [1]:
import os
import dotenv as env
from pydantic import BaseModel
from openai import OpenAI
import json
import pandas
import pprint as pp
import sys

env.load_dotenv('../../keys/keys.env')

True

In [2]:
class HeadlineClassification(BaseModel):
    title: str                          #Do not change the name of this field as OpenAI is dumb. If you keep it as inputs, then only it returns the original text sent to it.
                                        # Will have to carefully write wrappers to ensure that the outcome df to be used in the final output has the right headers. 
    classification: bool
    explanation: str

In [3]:
class listClassifications(BaseModel):
    output: list[HeadlineClassification]

In [4]:
class Recommendations(BaseModel):
    reco_date: str
    analyst: str
    house: str
    stock_name: str
    instrument_type: str
    ticker: str
    current_value: float
    target_value: float
    sl_value: float
    timeframe: str

In [5]:
class listRecommendations(BaseModel):
    output: list[Recommendations]

In [6]:
class scrips(BaseModel):
    company_name: str
    ticker: str

In [7]:
class listScrips(BaseModel):
    output: list[scrips]

In [8]:
class OpenAIModels:
    #__client: 
    __model_name = 'gpt-4.1-nano-2025-04-14'
    __prompt_file_path = '../../prompts/open_ai_prompts.json'

    def __init__(self):
        #self.__model_name = 'gpt-4.1-nano-2025-04-14'
        #self.__prompt_file_path = '../prompts/open_ai_prompts.json'
        self.__client = OpenAI(api_key=os.environ.get('OPENAI_API_KEY'))
    
    def __getPromptFromFile(self, type: str) -> str:
        with open(OpenAIModels.__prompt_file_path, 'r') as file:
            data = file.read()
        parsed_data = json.loads(data)
        prompt = parsed_data.get(type)
        return prompt
    
    def classify_headlines(self, input: pandas.Series, silent_mode=True) -> str :
        if(silent_mode):
            original_stdout = sys.stdout   # save the original stdout
            sys.stdout = open(os.devnull, 'w')
        prompt = self.__getPromptFromFile('headlines_classifier') 
        print('Sending titles for classification to OpenAI model...')
        print(f'Input is of length: {len(input)}.')
        try:
            response = self.__client.responses.parse(
                        model=OpenAIModels.__model_name,
                        #service_tier='default',
                        temperature=1.0,
                        instructions=prompt, 
                        input=input.to_json(),
                        text_format=listClassifications
                    )
            return response.output_text
        except Exception as e:
            print(f'Open AI execution threw an exception: {e}')
            return None
        finally:
            if(silent_mode):
                sys.stdout.close()
                sys.stdout = original_stdout    #restore the original stdout, other wise it will fail other prints.
            #Since the textformat here is a class wrapper of lists
            # So the return response must be handled to strip the 'output' dictionary to get to the inner lists

    def gather_articles_susbtance(self, input: pandas.DataFrame, silent_mode=True) -> str:
        if(silent_mode):
            original_stdout = sys.stdout   # save the original stdout
            sys.stdout = open(os.devnull, 'w')
        
        ##TODO
        prompt = self.__getPromptFromFile('article_substance')
        print('Sending article to OpenAI for gathering recommendation table...')
        try:
            response = self.__client.responses.parse(
                        model=OpenAIModels.__model_name,
                        #service_tier='default',
                        temperature=1.0,
                        instructions=prompt, 
                        input=input.to_json(orient='records'),   #orint in this form as it is a dataframe and not a series.
                        text_format=listRecommendations   
                    )
            return response.output_text
        except Exception as e:
            print(f'Open AI execution thew an error: {e}')
            return None
        finally:
            if(silent_mode):
                sys.stdout.close()
                sys.stdout = original_stdout    #restore the original stdout, other wise it will fail other prints.

    def gather_tickers(self, input: pandas.Series, silent_mode=True) -> str:
        if(silent_mode):
            original_stdout = sys.stdout   # save the original stdout
            sys.stdout = open(os.devnull, 'w')
        
        ##TODO
        prompt = self.__getPromptFromFile('scrip_names')
        print('Sending list to OpenAI for gathering scrip names...')
        try:
            response = self.__client.responses.parse(
                        model=OpenAIModels.__model_name,
                        #service_tier='default',
                        temperature=1.0,
                        instructions=prompt, 
                        input=input.to_json(),
                        text_format=listScrips   
                    )
            return response.output_text
        except Exception as e:
            print(f'Open AI execution threw an error: {e}')
            return None
        finally:
            if(silent_mode):
                sys.stdout.close()
                sys.stdout = original_stdout    #restore the original stdout, other wise it will fail other prints.

In [9]:
# Tester

#model = OpenAIModels()
#hl = ['Write a one-sentence bedtime story about a unicorn', 
#        'Hello my man xx!', 
#        'Sumeet Bagadia recommended buying these seven stock to buy',
#        'OpenAI is inconsistent!',
#        ]
#hl_series = pandas.Series(hl)
#response = model.classify_headlines(hl_series)
#pp.pprint(json.loads(response)['output'])

In [10]:
#Tester

#import ipynb.fs.full.gather_hl_and_articles as reader

#rdr = reader.Livemint_reader()
#model = OpenAIModels()

#frame = rdr.read_article('https://www.livemint.com/market/stock-market-news/concor-to-dabur-india-jay-thakkar-suggests-three-stocks-to-buy-for-short-term-in-f-o-segment-11748414675124.html')
#response = model.gather_articles_susbtance(frame, silent_mode=False)
#pp.pprint(json.loads(response)['output'])

Sending article to OpenAI for gathering recommendation table...
[{'analyst': 'Jay Thakkar',
  'current_value': 765,
  'house': 'ICICI Securities',
  'instrument_type': 'Futures',
  'reco_date': '2025-05-28',
  'sl_value': 730,
  'stock_name': 'CONCOR',
  'target_value': 850,
  'ticker': 'CONCOR',
  'timeframe': 'June'},
 {'analyst': 'Jay Thakkar',
  'current_value': 900,
  'house': 'ICICI Securities',
  'instrument_type': 'Futures',
  'reco_date': '2025-05-28',
  'sl_value': 865,
  'stock_name': 'Tata Chemicals',
  'target_value': 950,
  'ticker': 'N/A',
  'timeframe': 'June'},
 {'analyst': 'Jay Thakkar',
  'current_value': 480,
  'house': 'ICICI Securities',
  'instrument_type': 'Futures',
  'reco_date': '2025-05-28',
  'sl_value': 470,
  'stock_name': 'Dabur India',
  'target_value': 525,
  'ticker': 'N/A',
  'timeframe': 'June'}]
